<a href="https://colab.research.google.com/github/Nikhil-Chavanke-21/Explainability-of-ASR/blob/main/speaker_command_phone_data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# installing the requirements
!pip install datasets
!pip install torchaudio
!pip install transformers

In [ ]:
# importing the libraries
import math
import torch
import pickle
import torchaudio
import numpy as np
import soundfile as sf
from datasets import load_dataset
from transformers import Wav2Vec2Processor, Wav2Vec2Model

In [ ]:
# Downloading and loading the Wav2Vec2Processor & Wav2Vec2Model Large Size
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

In [ ]:
# For generating Speaker Classification data in to pickle file

# only 25 speakers out of 600 are taken into for the classification task
speakers={'FKSR0': 0, 'FMMH0': 1, 'MTPG0': 2, 'MTAT1': 3, 'MMDM1': 4, 'FPJF0': 5, 'FMML0': 6, 'MDLF0': 7, 'MRRK0': 8, 'MSES0': 9, 'MJES0': 10, 'FCJS0': 11, 'MCEM0': 12, 'FCAJ0': 13, 'FPMY0': 14, 'FKLC0': 15, 'FJSK0': 16, 'FNKL0': 17, 'FETB0': 18, 'FCLT0': 19, 'FLAS0': 20 , 'MJRF0': 21, 'MJDM0': 22, 'MDLD0': 23, 'MJDC0': 24}

# reading wav file to get speech frames as a array
def map_to_array(batch):
  speech, _ = sf.read(batch["file"])
  batch["speech"] = speech
  return batch

# getting train, test split of the dataset
train_dataset=load_dataset("timit_asr",split='train')
test_dataset=load_dataset("timit_asr",split='test')

# array from wav file
train_dataset = train_dataset.map(map_to_array)
test_dataset = test_dataset.map(map_to_array)

n=len(train_dataset)+len(test_dataset)
permutation=np.random.permutation(n)

# Opening files to pickle dump all attention layer representations for further classification
input_file=open("x.txt", "wb")
target_file=open("y.txt", "wb")
print('total:',n)
count=0
y=[]

# permutation for shuffling
for i in permutation:
  print('count',count)
  count+=1
  data=0
  # mixing train and test data
  if i<len(train_dataset):
    data=train_dataset[int(i)]
  else:
    data=train_dataset[int(i)-len(train_dataset)]
  
  # taking only 25 speakers data
  if data['speaker_id'] not in speakers:
    continue

  # preprocessing using Wav2Vec2Processor
  x=processor(data["speech"], return_tensors="pt", sampling_rate=16000).input_values
  # Generating data representations using Wav2Vec2Model
  x=model.forward(x,output_hidden_states=True)
  x=list(x.hidden_states)

  # average pooling accross timeframes
  for i, a in enumerate(x):
    x[i]=torch.sum(a,(0,1))
  # dumping all hidden activations
  pickle.dump(x, input_file)
  # getting speaker class index
  y.append(speakers[data['speaker_id']])

# dumping the class indices
pickle.dump(y,target_file)

In [ ]:
# For generating Speaker Classification data in to pickle file

# only 25 speakers out of 600 are taken into for the classification task
speakers={'FKSR0': 0, 'FMMH0': 1, 'MTPG0': 2, 'MTAT1': 3, 'MMDM1': 4, 'FPJF0': 5, 'FMML0': 6, 'MDLF0': 7, 'MRRK0': 8, 'MSES0': 9, 'MJES0': 10, 'FCJS0': 11, 'MCEM0': 12, 'FCAJ0': 13, 'FPMY0': 14, 'FKLC0': 15, 'FJSK0': 16, 'FNKL0': 17, 'FETB0': 18, 'FCLT0': 19, 'FLAS0': 20 , 'MJRF0': 21, 'MJDM0': 22, 'MDLD0': 23, 'MJDC0': 24}
phonemes={'y': 0, 'n': 1, 'dx': 2, 'er': 3, 'eh': 4, 'el': 5, 'iy': 6, 'ow': 7, 'ah': 8, 'hv': 9, 'dcl': 10, 'g': 11, 'b': 12, 'p': 13, 'r': 14, 'm': 15, 'en': 16, 'z': 17, 'jh': 18, 'k': 19, 'ey': 20, 'kcl': 21, 'em': 22, 's': 23, 'ih': 24, 'axr': 25, 'd': 26, 'th': 27, 'f': 28, 'h#': 29, 'bcl': 30, 'ax-h': 31, 't': 32, 'ay': 33, 'w': 34, 'epi': 35, 'ux': 36, 'ix': 37, 'ae': 38, 'uh': 39, 'v': 40, 'ch': 41, 'hh': 42, 'ax': 43, 'tcl': 44, 'dh': 45, 'sh': 46, 'nx': 47, 'ao': 48, 'ng': 49, 'aw': 50, 'gcl': 51, 'l': 52, 'uw': 53, 'aa': 54, 'q': 55, 'eng': 56, 'oy': 57, 'pcl': 58, 'zh': 59, 'pau': 60}

# reading wav file to get speech frames as a array
def map_to_array(batch):
  speech, _ = sf.read(batch["file"])
  batch["speech"] = speech
  return batch

# getting train, test split of the dataset
train_dataset=load_dataset("timit_asr",split='train')
test_dataset=load_dataset("timit_asr",split='test')

# array from wav file
train_dataset = train_dataset.map(map_to_array)
test_dataset = test_dataset.map(map_to_array)

n=len(train_dataset)+len(test_dataset)
permutation=np.random.permutation(n)

# Opening files to pickle dump all attention layer representations for further classification
input_file=open("x.txt", "wb")
target_file=open("y.txt", "wb")
print('total:',n)
count=0
y=[]

# permutation for shuffling
for i in permutation:
  print('count',count)
  count+=1
  data=0
  # mixing train and test data
  if i<len(train_dataset):
    data=train_dataset[int(i)]
  else:
    data=train_dataset[int(i)-len(train_dataset)]
  
  # taking only 25 speakers data
  if data['speaker_id'] not in speakers:
    continue

  # preprocessing using Wav2Vec2Processor
  x=processor(data["speech"], return_tensors="pt", sampling_rate=16000).input_values
  phones=data['phonetic_detail']
  # Generating data representations using Wav2Vec2Model
  x=model.forward(x,output_hidden_states=True)
  x=list(x.hidden_states)

  # dividing the speech frames into groups corresponding to phone start and stop frame
  for j in range(len(phones['start'])):
    sampling=x[0].size(1)/len(data["speech"])
    start=math.floor(sampling*phones['start'][j])
    stop=math.floor(sampling*phones['stop'][j])
    phone=phones['utterance'][j]

    temp=[]

    # average pooling accross timeframes
    for i, a in enumerate(x):
      temp.append(torch.sum(a[0,start:stop,:],(0)))
    # dumping all hidden activations
    pickle.dump(temp, input_file)
    # getting phone label
    y.append(phonemes[phone])

# dumping the class indices
pickle.dump(y,target_file)

In [ ]:
# Loading the Speech Commands Dataset
speech_commands=torchaudio.datasets.SPEECHCOMMANDS('/',download=True)

# 35 Commands in the dataset
commands_labels=['backward', 'bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'follow', 'forward', 'four', 'go', 'happy', 'house', 'learn', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'visual', 'wow', 'yes', 'zero']
counts={'backward': 0, 'bed': 0, 'bird': 0, 'cat': 0, 'dog': 0, 'down': 0, 'eight': 0, 'five': 0, 'follow': 0, 'forward': 0, 'four': 0, 'go': 0, 'happy': 0, 'house': 0, 'learn': 0, 'left': 0, 'marvin': 0, 'nine': 0, 'no': 0, 'off': 0, 'on': 0, 'one': 0, 'right': 0, 'seven': 0, 'sheila': 0, 'six': 0, 'stop': 0, 'three': 0, 'tree': 0, 'two': 0, 'up': 0, 'visual': 0, 'wow': 0, 'yes': 0, 'zero': 0}
class_ind={'backward': 0, 'bed': 1, 'bird': 2, 'cat': 3, 'dog': 4, 'down': 5, 'eight': 6, 'five': 7, 'follow': 8, 'forward': 9, 'four': 10, 'go': 11, 'happy': 12, 'house': 13, 'learn': 14, 'left': 15, 'marvin': 16, 'nine': 17, 'no': 18, 'off': 19, 'on': 20, 'one': 21, 'right': 22, 'seven': 23, 'sheila': 24, 'six': 25, 'stop': 26, 'three': 27, 'tree': 28, 'two': 29, 'up': 30, 'visual': 31, 'wow': 32, 'yes': 33, 'zero': 34}

# Opening files to pickle dump all attention layer representations for further classification
input_file=open("x.txt", "wb")
target_file=open("y.txt", "wb")

y=[]
count=0
# Getting hidden activations for all layers for each data instance
for i, x in enumerate(speech_commands):
  # 50 instance for each command taken
  if counts[x[2]]>=50:
    continue
  counts[x[2]]=counts[x[2]]+1
  
  # preprocessing using Wav2Vec2Processor
  input_values=processor(x[0],sampling_rate=16000, return_tensors="pt").input_values
  # Generating data representations using Wav2Vec2Model
  a=model.forward(input_values[0],output_hidden_states=True)
  source=[]
  # average pooling accross timeframes
  for t in a.hidden_states:
    source.append(torch.mean(t,(0,1)))
  # dumping all hidden activations
  pickle.dump(source,input_file)
  y.append(class_ind[x[2]])
  print(count)
  count+=1
# dumping the class indices
pickle.dump(y,target_file)